In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import requests,json
import sys
import pandas as pd
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO
from dateutil.relativedelta import relativedelta
import calendar
from datetime import date, datetime, time, timedelta
import matplotlib.pyplot as plt

url="https://dataapi.joinquant.com/apis"

def get_token():
    body={       
        "method": "get_token",
        "mob": "13983706223",  #mob是申请JQData时所填写的手机号
        "pwd": "706223",  #Password为聚宽官网登录密码，新申请用户默认为手机号后6位
    }
    return requests.post(url, data = json.dumps(body)).text

def get_data(response):   # 数据处理函数,处理csv字符串函数
    '''格式化数据为DataFrame'''
    return pd.read_csv(StringIO(response.text), parse_dates=True)

token = get_token()

In [2]:
def http_get_bars_period(code,unit,start_date,end_date=None,fq_ref_date=None):
    body ={
        "method": "get_price_period",
        "token": token,
        "code": code,
        "unit": unit,
        "date": start_date,    
        "end_date": end_date,
        "fq_ref_date": fq_ref_date
    }
#     print(requests.post(url, data = json.dumps(body)).text)
    return  get_data(requests.post(url, data = json.dumps(body))).set_index('date')

In [3]:
def http_get_index_stocks(code,date=None):
    body ={
    "method": "get_index_stocks",
    "token": token,
    "code": code,
    "date":date
    }
    return requests.post(url, data = json.dumps(body)).text.split('\n')



In [4]:
#000300.XSHG => CSI300
#000905.XSHG => CSI500
universe = http_get_index_stocks('000905.XSHG')

In [5]:
history = {}
start_date = '2020-01-01'
for ticker in universe:
    history[ticker] = http_get_bars_period(ticker,'1d',start_date)

In [6]:
import pandas as pd
open_df = pd.DataFrame()
close_df = pd.DataFrame()
high_df = pd.DataFrame()
low_df = pd.DataFrame()

for ticker in history.keys():
    open_df[ticker] = history[ticker]['open']
    close_df[ticker] = history[ticker]['close']
    high_df[ticker] = history[ticker]['high']
    low_df[ticker] = history[ticker]['low']
        

In [7]:
close_open_vol = (close_df / open_df - 1).rolling(21).std().fillna(method='bfill')
close_open_vol;

In [8]:
high_low_vol = (high_df / low_df -1).rolling(21).std().fillna(method='bfill')
high_low_vol;

In [9]:
hc_ratio = (high_low_vol / close_open_vol).rolling(21).mean().replace([np.inf, -np.inf], np.nan)
hc_ranked = hc_ratio.rank(ascending=True, pct=True, axis=1)

In [11]:
hc_ranked.tail(5)

,000008.XSHE,000009.XSHE,000012.XSHE,000021.XSHE,000025.XSHE,000027.XSHE,000028.XSHE,000031.XSHE,000039.XSHE,000046.XSHE,...,603816.XSHG,603858.XSHG,603866.XSHG,603868.XSHG,603882.XSHG,603883.XSHG,603885.XSHG,603888.XSHG,603939.XSHG,603983.XSHG
2020-10-30,0.552,0.172,0.612,0.578,0.778,0.444,0.360,0.224,0.282,0.770,...,0.750,0.126,0.250,0.114,0.344,0.004,0.856,0.354,0.408,0.222
2020-11-02,0.566,0.188,0.632,0.584,0.802,0.434,0.340,0.178,0.322,0.760,...,0.758,0.130,0.248,0.090,0.348,0.004,0.886,0.310,0.432,0.244
2020-11-03,0.570,0.208,0.658,0.580,0.812,0.430,0.326,0.112,0.402,0.740,...,0.748,0.132,0.246,0.086,0.372,0.004,0.904,0.276,0.460,0.258
2020-11-04,0.564,0.236,0.674,0.566,0.822,0.392,0.320,0.068,0.512,0.732,...,0.704,0.126,0.228,0.090,0.394,0.004,0.928,0.230,0.494,0.252
2020-11-05,0.564,0.248,0.696,0.572,0.818,0.352,0.322,0.048,0.594,0.732,...,0.700,0.134,0.202,0.082,0.418,0.004,0.938,0.178,0.514,0.230


In [19]:
mypick = hc_ranked.sort_values(by='2020-11-05', axis=1, ascending=False)

In [32]:
mypick.iloc[:,0:10]

,600094.XSHG,600160.XSHG,600008.XSHG,600903.XSHG,000959.XSHE,600970.XSHG,002302.XSHE,600823.XSHG,002503.XSHE,600664.XSHG
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-10-30,0.994,1.000,0.998,0.992,0.996,0.984,0.990,0.974,0.986,0.976
2020-11-02,0.996,1.000,0.998,0.992,0.994,0.988,0.990,0.978,0.986,0.976
2020-11-03,0.996,1.000,0.998,0.992,0.994,0.988,0.990,0.982,0.986,0.980
2020-11-04,0.998,1.000,0.996,0.994,0.992,0.988,0.990,0.984,0.986,0.980


In [33]:
mypick.iloc[-1,0:10]

600094.XSHG    1.000
600160.XSHG    0.998
600008.XSHG    0.996
600903.XSHG    0.994
000959.XSHE    0.992
600970.XSHG    0.990
002302.XSHE    0.988
600823.XSHG    0.986
002503.XSHE    0.984
600664.XSHG    0.982
Name: 2020-11-05, dtype: float64